In [2]:
# 토큰 정보로드를 위한 라이브러리
# 설치: pip install python-dotenv
from dotenv import load_dotenv

# 토큰 정보로드
load_dotenv()

True

In [3]:
# 허깅페이스 모델/토크나이저를 다운로드 받을 경로
# (예시)

import os

# ./cache/ 경로에 다운로드 받도록 설정
os.environ["TRANSFORMERS_CACHE"] = "./cache/"
os.environ["HF_HOME"] = "./cache/"

In [4]:
from langchain.llms import HuggingFaceHub
import warnings

# 경고 메시지 무시
warnings.filterwarnings("ignore")

# HuggingFace Repository ID
# "KT-AI/midm-bitext-S-7B-inst-v1"
# "mistralai/Mixtral-8x7B-Instruct-v0.1"
repo_id = "mistralai/Mixtral-8x7B-Instruct-v0.1"

# HuggingFaceHub 객체 생성
llm = HuggingFaceHub(
    repo_id=repo_id,
    model_kwargs={"temperature": 0.1, "max_length": 2048},
    task="text-generation",  # 텍스트 생성
)


In [5]:
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

In [6]:

template = """Summarizes TEXT in simple bullet points ordered from most important to least important.
TEXT:
{text}

KeyPoints: """

prompt = PromptTemplate.from_template(template)

llm_chain = LLMChain(prompt=prompt, llm=llm)

text = """A Large Language Model (LLM) like me, ChatGPT, is a type of artificial intelligence (AI) model designed to understand, generate, and interact with human language. These models are "large" because they're built from vast amounts of text data and have billions or even trillions of parameters. Parameters are the aspects of the model that are learned from training data; they are essentially the internal settings that determine how the model interprets and generates language. LLMs work by predicting the next word in a sequence given the words that precede it, which allows them to generate coherent and contextually relevant text based on a given prompt. This capability can be applied in a variety of ways, from answering questions and composing emails to writing essays and even creating computer code. The training process for these models involves exposing them to a diverse array of text sources, such as books, articles, and websites, allowing them to learn language patterns, grammar, facts about the world, and even styles of writing. However, it's important to note that while LLMs can provide information that seems knowledgeable, their responses are generated based on patterns in the data they were trained on and not from a sentient understanding or awareness. The development and deployment of LLMs raise important considerations regarding accuracy, bias, ethical use, and the potential impact on various aspects of society, including employment, privacy, and misinformation. Researchers and developers continue to work on ways to address these challenges while improving the models' capabilities and applications."""

response = llm_chain.invoke(input=text)


In [7]:
print(response["text"])

Summarizes TEXT in simple bullet points ordered from most important to least important.
TEXT:
A Large Language Model (LLM) like me, ChatGPT, is a type of artificial intelligence (AI) model designed to understand, generate, and interact with human language. These models are "large" because they're built from vast amounts of text data and have billions or even trillions of parameters. Parameters are the aspects of the model that are learned from training data; they are essentially the internal settings that determine how the model interprets and generates language. LLMs work by predicting the next word in a sequence given the words that precede it, which allows them to generate coherent and contextually relevant text based on a given prompt. This capability can be applied in a variety of ways, from answering questions and composing emails to writing essays and even creating computer code. The training process for these models involves exposing them to a diverse array of text sources, suc

## 허깅페이스 파이프라인(HuggingFace Pipeline)

__Hugging Face Local Pipelines__

뭔가 세부 옵션 튜닝 느낌?

HuggingFacePipeline 클래스를 통해 Hugging Face 모델을 로컬에서 실행할 수 있습니다.

Hugging Face Model Hub는 온라인 플랫폼에서 120,000개 이상의 모델, 20,000개의 데이터셋, 50,000개의 데모 앱(Spaces)을 호스팅하며, 모두 오픈 소스이고 공개적으로 사용 가능

이러한 모델은 LangChain에서 이 로컬 파이프라인 래퍼를 통해 호출하거나, HuggingFaceHub 클래스를 통해 호스팅된 추론 엔드포인트를 호출하여 사용할 수 있습니다.

사용하기 위해서는 PyTorch와 함께 Python 패키지 transformers가 설치되어 있어야 합니다.

또한, 보다 메모리 효율적인 attention 구현을 위해 xformer를 설치할 수도 있습니다.

In [8]:
!pip install --upgrade --quiet  transformers --quiet

In [9]:
# 허깅페이스 모델/토크나이저를 다운로드 받을 경로
# (예시)
import os

# ./cache/ 경로에 다운로드 받도록 설정
os.environ["TRANSFORMERS_CACHE"] = "./cache/"
os.environ["HF_HOME"] = "./cache/"

### Model Loading

모델은 from_model_id 메서드를 사용하여 모델 매개변수를 지정함으로써 로드할 수 있습니다.

- HuggingFacePipeline 클래스를 사용하여 Hugging Face의 사전 학습된 모델을 로드합니다.
- from_model_id 메서드를 사용하여 beomi/llama-2-ko-7b 모델을 지정하고, 작업을 "text-generation"으로 설정합니다.
- pipeline_kwargs 매개변수를 사용하여 생성할 최대 토큰 수를 10으로 제한합니다.
- 로드된 모델은 hf 변수에 할당되며, 이를 통해 텍스트 생성 작업을 수행할 수 있습니다.
- 사용한 모델: https://huggingface.co/beomi/llama-2-ko-7b

In [12]:
from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline

In [13]:
# HuggingFace 모델을 다운로드 받습니다.
hf = HuggingFacePipeline.from_model_id(
    model_id="beomi/llama-2-ko-7b",  # 사용할 모델의 ID를 지정합니다.
    task="text-generation",  # 수행할 작업을 지정합니다. 여기서는 텍스트 생성입니다.
    # 파이프라인에 전달할 추가 인자를 설정합니다. 여기서는 생성할 최대 토큰 수를 10으로 제한합니다.
    pipeline_kwargs={"max_new_tokens": 512},
)

# 기존의 transformers pipeline을 직접 전달하여 로드할 수도 있습니다.

model-00008-of-00015.safetensors:   0%|          | 0.00/967M [00:00<?, ?B/s]

model-00009-of-00015.safetensors:   0%|          | 0.00/967M [00:00<?, ?B/s]

model-00010-of-00015.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

model-00011-of-00015.safetensors:   0%|          | 0.00/944M [00:00<?, ?B/s]

model-00012-of-00015.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

model-00013-of-00015.safetensors:   0%|          | 0.00/967M [00:00<?, ?B/s]

model-00014-of-00015.safetensors:   0%|          | 0.00/742M [00:00<?, ?B/s]

model-00015-of-00015.safetensors:   0%|          | 0.00/380M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/15 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/159 [00:00<?, ?B/s]

HuggingFacePipeline을 사용하여 텍스트 생성 모델을 구현합니다.

- AutoTokenizer와 AutoModelForCausalLM을 사용하여, "beomi/llama-2-ko-7b 모델"과 "토크나이저"를 로드합니다.
- pipeline 함수를 사용하여 "text-generation" 파이프라인을 생성하고, "모델과 토크나이저"를 설정합니다. 최대 생성 토큰 수는 10으로 제한합니다.
- HuggingFacePipeline 클래스를 사용하여 hf 객체를 생성하고, 생성된 파이프라인을 전달합니다.

이렇게 생성된 hf 객체를 사용하여 주어진 프롬프트에 대한 텍스트 생성을 수행할 수 있습니다.

In [14]:
from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

model_id = "beomi/llama-2-ko-7b"  # 사용할 모델의 ID를 지정합니다.

# 지정된 모델의 토크나이저를 로드합니다.
tokenizer = AutoTokenizer.from_pretrained(model_id)  

 # 지정된 모델을 로드합니다.
model = AutoModelForCausalLM.from_pretrained(model_id) 

# 텍스트 생성 파이프라인을 생성하고, 최대 생성할 새로운 토큰 수를 10으로 설정합니다.
pipe = pipeline("text-generation", 
                model=model,
                tokenizer=tokenizer, 
                max_new_tokens=512)

# HuggingFacePipeline 객체를 생성하고, 생성된 파이프라인을 전달합니다.
hf = HuggingFacePipeline(pipeline=pipe)


Loading checkpoint shards:   0%|          | 0/15 [00:00<?, ?it/s]

### Create Chain

모델이 메모리에 로드되면 프롬프트와 함께 구성하여 체인을 형성할 수 있습니다.

- PromptTemplate 클래스를 사용하여 질문과 답변 형식을 정의하는 프롬프트 템플릿을 생성합니다.
- prompt 객체와 hf 객체를 파이프라인으로 연결하여 chain 객체를 생성합니다.
- chain.invoke() 메서드를 호출하여 주어진 질문에 대한 답변을 생성하고 출력합니다.

In [ ]:
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

# 답변은 한국어로 하기 
template = """Answer the following question in Korean.
#Question: 
{question}

#Answer: """  # 질문과 답변 형식을 정의하는 템플릿
prompt = PromptTemplate.from_template(template)  # 템플릿을 사용하여 프롬프트 객체 생성

# 프롬프트와 언어 모델을 연결하여 체인 생성
chain = prompt | hf | StrOutputParser()

question = "대한민국의 수도는 어디야?"  # 질문 정의

print(
    chain.invoke({"question": question})
)  # 체인을 호출하여 질문에 대한 답변 생성 및 출력


### GPU Inference

GPU에서 실행할 때는 device=n 매개변수를 지정하여 모델을 특정 디바이스에 배치할 수 있습니다.

기본값은 -1로, CPU에서 추론을 수행합니다.

여러 개의 GPU가 있거나 모델이 단일 GPU에 비해 너무 큰 경우에는 device_map="auto"를 지정할 수 있습니다.

이 경우 Accelerate 라이브러리가 필요하며, 모델 가중치를 어떻게 로드할지 자동으로 결정하는 데 사용됩니다.

주의: device와 device_map은 함께 지정해서는 안 되며, 예기치 않은 동작을 유발할 수 있습니다.

- HuggingFacePipeline을 사용하여 gpt2 모델을 로드하고, device 매개변수를 0으로 설정하여 GPU에서 실행되도록 합니다.
- pipeline_kwargs 매개변수를 사용하여 생성할 최대 토큰 수를 10으로 제한합니다.
- prompt와 gpu_llm을 파이프라인으로 연결하여 gpu_chain을 생성합니다.
- gpu_chain.invoke() 메서드를 호출하여 주어진 질문에 대한 답변을 생성하고 출력합니다.

In [ ]:
gpu_llm = HuggingFacePipeline.from_model_id(
    model_id="beomi/llama-2-ko-7b",  # 사용할 모델의 ID를 지정합니다.
    task="text-generation",  # 수행할 작업을 설정합니다. 여기서는 텍스트 생성입니다.
    device=0,       # 사용할 GPU 디바이스 번호를 지정합니다. "auto"로 설정하면 accelerate 라이브러리를 사용합니다.
    pipeline_kwargs={"max_new_tokens": 64},   # 파이프라인에 전달할 추가 인자를 설정합니다. 여기서는 생성할 최대 토큰 수를 10으로 제한합니다.
)

gpu_chain = prompt | gpu_llm  # prompt와 gpu_llm을 연결하여 gpu_chain을 생성합니다.

# 프롬프트와 언어 모델을 연결하여 체인 생성
gpu_chain = prompt | gpu_llm | StrOutputParser()

question = "대한민국의 수도는 어디야?"  # 질문 정의

# 체인을 호출하여 질문에 대한 답변 생성 및 출력
print(gpu_chain.invoke({"question": question}))


### Batch GPU Inference

GPU 장치에서 실행하는 경우, 배치 모드로 GPU에서 추론을 실행할 수 있습니다.

- HuggingFacePipeline을 사용하여 beomi/llama-2-ko-7b 모델을 로드하고, GPU에서 실행되도록 설정합니다.
- gpu_llm을 생성할 때 batch_size를 2로 설정하고, temperature를 0으로, max_length를 64로 설정합니다.
- prompt와 gpu_llm을 파이프라인으로 연결하여 gpu_chain을 생성하고, 종료 토큰을 "\n\n"로 설정합니다.
- gpu_chain.batch()를 사용하여 questions의 질문들에 대한 답변을 병렬로 생성합니다.
- 생성된 답변을 반복문을 통해 출력합니다.

In [ ]:
gpu_llm = HuggingFacePipeline.from_model_id(
    model_id="beomi/llama-2-ko-7b",  # 사용할 모델의 ID를 지정합니다.
    task="text-generation",  # 수행할 작업을 설정합니다.
    device=0,  # GPU 디바이스 번호를 지정합니다. -1은 CPU를 의미합니다.
    batch_size=2,  # 배치 크기s를 조정합니다. GPU 메모리와 모델 크기에 따라 적절히 설정합니다.
    model_kwargs={
        "temperature": 0,
        "max_length": 256,
    },  # 모델에 전달할 추가 인자를 설정합니다.
)

# 프롬프트와 언어 모델을 연결하여 체인을 생성합니다.
gpu_chain = prompt | gpu_llm.bind(stop=["\n\n"])

questions = []
for i in range(4):
    # 질문 리스트를 생성합니다.
    questions.append({"question": f"숫자 {i} 이 한글로 뭐에요?"})

answers = gpu_chain.batch(questions)  # 질문 리스트를 배치 처리하여 답변을 생성합니다.
for answer in answers:
    print(answer)  # 생성된 답변을 출력합니다.
